# Parse PubMed data in order to get plain text of relevant papers

Through the URLs below, PubMed data can be ontained interactively. The idea here is to select papers based on keywords (and perhaps dates, ...) in a first query that results in PMC IDs. After that, the full text of those publications can be ontained one-by-one.

These texts can be analyzed by scispacy language modesl that include POS tagging for biological/scienti

In [1]:
import pandas as pd
import scispacy
import spacy

from utils import perform_query, extract_clean, retrieve_paper


## Obtain and process data
Steps:
1. Define a query to search papers
2. Call API with that query to obtain all PMC IDs for that query
3. Read the full content of tyhose papers one by one
4. Pre-process the full text
5. Run language model on clean text of body of paper

__TODO:__ indicate the PMCID if no body is retrieved. Case where only the abstract is available?  
__TODO:__ add a parameter to the query 

In [ ]:
query = 'tomato%20trichome'
#query = 'tomato'

IDs = perform_query(query, page_size=1000)

all_papers = []
for ID in IDs:
    content = retrieve_paper(ID)
    cleantext = extract_clean(content)
    all_papers.append(cleantext)

Can't find the body, returning nothing!
Can't find the body, returning nothing!
Can't find the body, returning nothing!
Can't find the body, returning nothing!
Can't find the body, returning nothing!
Can't find the body, returning nothing!
Can't find the body, returning nothing!
Can't find the body, returning nothing!
Can't find the body, returning nothing!
Can't find the body, returning nothing!


In [ ]:
len(IDs)

In [ ]:
# how to load: en_ner_bionlp13cg_md-0.4.0.tar.gz
nlp = spacy.load(name = "en_ner_bionlp13cg_md")

In [ ]:
docs = [nlp(text) for text in all_papers]

In [ ]:
len(docs)

This section returns the word, its start and end position of a word together with its label. 

In [ ]:
#for ent in docs[2].ents:
#    print(f"{ent.text:20s}, {ent.start_char:6}, {ent.end_char:6}, {ent.label_:20s}")

In [ ]:
for ent in docs[2].ents:
    # Gene symbols appear in the text as complete upper case words
    if "GENE" in ent.label_ and ent.text.isupper() and len(ent.text) > 2:
        print(f"{ent.text:20s}, {ent.start_char:6}, {ent.end_char:6}, {ent.label_:20s}")

# Calculate distance matrix between terms based on the whole document

## Compute a distance matrix from the terms start positions

## 

# Combine distance information with syntax sentence syntax information

__Goal:__ give a direction or a sign to each edge in the network. For instance:   
- Is gene A positively or negatively regulated by gene B?   
- Is the level/activity of gene A neg/pos influenced by gene B?  
This information should come from verbs occuring in sentences where A and B occur together.   

Steps:   
- label the verbs manually. 
- train a model to assign a positive/negative regulation.  

In [ ]:
for token in docs[2][100:110]:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)